In [1]:
import os
google_api_key = os.getenv("MY_GOOGLE_AI_STUDIO_KEY")  # Fetch secret from env variable

In [2]:
# Write code to get gemini models
import google.generativeai as genai
genai.configure(api_key = google_api_key)
model = genai.GenerativeModel('gemini-2.0-flash-exp')
model_getFunctions = genai.GenerativeModel('gemini-2.0-flash-exp')

In [3]:
def get_files_in_folder(root_dir):    
    """Recursively scan a folder and return list of files with full paths"""
    import os
    file_list = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            full_path = os.path.join(dirpath, filename)
            file_list.append(full_path)
    return file_list

In [4]:
def get_file_types(file_list):    
    """Return dictionary mapping filenames to their file extensions."""
    import os
    type_dict = {}
    for file_path in file_list:
        # Split the file extension from the path
        _, ext = os.path.splitext(file_path)
        # Remove leading dot and convert to lowercase for consistency
        clean_ext = ext.lower().lstrip('.') if ext else 'no_extension'
        type_dict[file_path] = clean_ext
    return type_dict

In [5]:
def organize_files(type_dict):
    """Organize files into folders based on their type in their respective directories."""
    import os
    import shutil
    
    for file_path, file_type in type_dict.items():
        # Get parent directory and filename
        parent_dir = os.path.dirname(file_path)
        filename = os.path.basename(file_path)
        
        # Create target directory name (pluralize for readability)
        target_dir_name = f"{file_type}s"  # Example: 'pdf' becomes 'pdfs'
        target_dir = os.path.join(parent_dir, target_dir_name)
        
        # Create target directory if it doesn't exist
        os.makedirs(target_dir, exist_ok=True)
        
        # Construct full target path
        target_path = os.path.join(target_dir, filename)
        
        # Only move if source and target paths are different
        if file_path != target_path:
            try:
                shutil.move(file_path, target_path)
                print(f"Moved: {filename} -> {target_dir_name}/")
            except Exception as e:
                print(f"Error moving {filename}: {str(e)}")


In [6]:
functions = '''
def get_files_in_folder(root_dir):    
    """Recursively scan a folder and return list of files with full paths""" 
    
def get_file_types(file_list):    
    """Return dictionary mapping filenames to their file extensions."""

def organize_files(type_dict):
    """Organize files into folders based on their type in their respective directories."""
  
    
    
'''

In [14]:
import gradio as gr

def process_input(option, text):
    if not option:
        return "Please select an option", "from the dropbox"
    
    prompt = f'''You are a AI assistant. Your job is to break a task into smaller 
    tasks that could be executed as code. Do not write the code, but just list the tasks as steps.
    Here is your task - {option}'''

    response = model.generate_content(prompt)    
    output1 = response.text

    prompt2 = f'''You are a AI assistant. You are given some steps. 
    Your job is to get the function calls for these steps from the list of functions below. 
    Give me an executable python code that uses these functions. Just make the function calls. 
    Do not write any code within these functions. Do not strictly write any other code apart from the function calls. 
    However you are allowed to save the output of these functions into variables and use them as input parameters to other functions. 
    You shall only use the functions mentioned here and you shall not write your own code. 
    Your code should strictly not use any other functions apart from functions mentioned below. 
    Here are your steps - {response.text} ; and
    here are your functions - {functions}'''

    response2 = model_getFunctions.generate_content(prompt2)

    
    output2 = response2.text
    return output1, output2

def handle_execute(code):
    model_formatCode = genai.GenerativeModel('gemini-2.0-flash-exp')
    if not code:
        print("Empty String - nothing to be executed")
    else:
        # Split into lines, remove first and last, then join back
        lines = code.splitlines()
        cleaned_code = "\n".join(lines[1:-1])
        print(cleaned_code)
        exec(cleaned_code)
        
with gr.Blocks() as demo:
    gr.Markdown("# EPAI Capstone - Agents")
    gr.Markdown('''This capstone demonstrate how agents work internally. This project accepts a user input, 
    breaks them into smaller tasks. Chooses from a predefined set of functions(tools) that correspond to these smaller tasks, 
    and,  finally executes all the functions. \n\n Here are the set of available functions(tools):\n
    def get_files_in_folder(root_dir) \n
    def get_file_types(file_list) \n
    def organize_files(type_dict) ''')
    
    with gr.Column():
        dropdown = gr.Dropdown(choices=['''scans a folder called MyDownlads, identify file types and 
move them into categorized folders (e.g., PDFs, images, code files, etc)'''], label="Select an option", interactive=True)
        text_input = gr.Textbox(label="Or enter your text", interactive=True, visible=False)        
        input_button = gr.Button("Submit")
    output1 = gr.Textbox(label="Your task broken into smaller text")
    output2 = gr.Textbox(label="Functions corresponding to your task from predefined toolbox")
    execute_button = gr.Button("Execute", visible=False)

    def handle_submit(option, text):
        out1, out2 = process_input(option, text)
        return out1, out2, gr.update(visible=True)
    
    input_button.click(handle_submit, inputs=[dropdown, text_input], 
                       outputs=[output1, output2, execute_button])
    
    execute_button.click(handle_execute, inputs=[output2])


demo.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.40.1, however version 4.44.1 is available, please upgrade.
--------
source_folder = "MyDownloads"
extension_mapping = {".pdf": "PDFs", ".jpg": "Images", ".py": "Code"}

file_list = get_files_in_folder(source_folder)
type_dict = get_file_types(file_list)
organize_files(type_dict)
Moved: RyanHoliday15Books.pdf -> pdfs/
